# Fourth Assignment Solution

## Image Rotation

In [1]:
import cv2
import numpy as np

def rotate_image(image, angle, interpolation=cv2.INTER_LINEAR):
    """
    Rotate an input image by a specified angle.

    Parameters:
        image: numpy.ndarray
            Input image.
        angle: float
            Angle in degrees by which to rotate the image.
        interpolation: int, optional
            Interpolation method to use. Default is cv2.INTER_LINEAR.

    Returns:
        numpy.ndarray
            Rotated image.
    """
    # Get image dimensions
    height, width = image.shape[:2]

    # Calculate rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)

    # Perform rotation
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), flags=interpolation)

    return rotated_image

# Load sample image
image = cv2.imread('logo.png')

# Test rotation by 45 degrees with bilinear interpolation
rotated_image_bilinear = rotate_image(image, 45, interpolation=cv2.INTER_LINEAR)

# Test rotation by 90 degrees with nearest neighbor interpolation
rotated_image_nearest = rotate_image(image, 90, interpolation=cv2.INTER_NEAREST)


## Image Cropping

In [2]:
def crop_image(image, x, y, width, height):
    """
    Crop an input image to a specified region of interest (ROI).

    Parameters:
        image: numpy.ndarray
            Input image.
        x: int
            X-coordinate of the top-left corner of the ROI.
        y: int
            Y-coordinate of the top-left corner of the ROI.
        width: int
            Width of the ROI.
        height: int
            Height of the ROI.

    Returns:
        numpy.ndarray
            Cropped image.
    """
    cropped_image = image[y:y+height, x:x+width]
    return cropped_image

# Load sample image
image = cv2.imread('logo.png')

# Define ROI coordinates and dimensions
x, y, width, height = 100, 100, 300, 200

# Crop image to the specified ROI
cropped_image = crop_image(image, x, y, width, height)

## Affine Transformation

In [3]:
def translate_image(image, tx, ty):
    """
    Translate (shift) an input image by the specified translation factors.

    Parameters:
        image: numpy.ndarray
            Input image.
        tx: int
            Translation factor along the x-axis.
        ty: int
            Translation factor along the y-axis.

    Returns:
        numpy.ndarray
            Translated image.
    """
    rows, cols = image.shape[:2]
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    translated_image = cv2.warpAffine(image, translation_matrix, (cols, rows))
    return translated_image

def scale_image(image, scale_x, scale_y):
    """
    Scale an input image by the specified scale factors.

    Parameters:
        image: numpy.ndarray
            Input image.
        scale_x: float
            Scale factor along the x-axis.
        scale_y: float
            Scale factor along the y-axis.

    Returns:
        numpy.ndarray
            Scaled image.
    """
    scaled_image = cv2.resize(image, None, fx=scale_x, fy=scale_y)
    return scaled_image

def shear_image(image, shear_x, shear_y):
    """
    Shear an input image by the specified shear factors.

    Parameters:
        image: numpy.ndarray
            Input image.
        shear_x: float
            Shear factor along the x-axis.
        shear_y: float
            Shear factor along the y-axis.

    Returns:
        numpy.ndarray
            Sheared image.
    """
    rows, cols = image.shape[:2]
    shear_matrix = np.float32([[1, shear_x, 0], [shear_y, 1, 0]])
    sheared_image = cv2.warpAffine(image, shear_matrix, (cols, rows))
    return sheared_image

def reflect_image(image, axis):
    """
    Reflect (flip) an input image along the specified axis.

    Parameters:
        image: numpy.ndarray
            Input image.
        axis: int
            Axis along which to reflect the image. 0 for vertical, 1 for horizontal.

    Returns:
        numpy.ndarray
            Reflected image.
    """
    reflected_image = cv2.flip(image, axis)
    return reflected_image

# Load sample image
image = cv2.imread('logo.png')

# Test individual transformations
translated_image = translate_image(image, 50, 50)
scaled_image = scale_image(image, 0.5, 0.5)
sheared_image = shear_image(image, 0.2, 0.2)
reflected_vertical_image = reflect_image(image, 0)
reflected_horizontal_image = reflect_image(image, 1)

## Perspective Transformation

In [4]:
def perspective_transform(image, src_points, dst_points):
    """
    Perform perspective transformation on an input image to correct distortions caused by the viewpoint of the camera.

    Parameters:
        image: numpy.ndarray
            Input image.
        src_points: list of tuples
            List of four source points defining the region of interest (ROI) in the input image.
        dst_points: list of tuples
            List of four destination points defining the desired perspective-transformed region.

    Returns:
        numpy.ndarray
            Perspective-transformed image.
    """
    # Convert points to numpy arrays
    src_points = np.array(src_points, dtype=np.float32)
    dst_points = np.array(dst_points, dtype=np.float32)

    # Calculate perspective transform matrix
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Apply perspective transform
    transformed_image = cv2.warpPerspective(image, perspective_matrix, (image.shape[1], image.shape[0]))

    return transformed_image

# Load sample image
image = cv2.imread('logo.png')

# Define source points (ROI) and destination points for perspective transformation
src_points = [(150, 150), (450, 150), (450, 350), (150, 350)]
dst_points = [(200, 100), (400, 100), (400, 300), (200, 300)]

# Perform perspective transformation
transformed_image = perspective_transform(image, src_points, dst_points)

## Color Indexing

In [5]:
from sklearn.cluster import KMeans

def index_colors(image, k):
    """
    Index colors in an image using k-means clustering.

    Parameters:
        image: numpy.ndarray
            Input image.
        k: int
            Number of clusters (colors) to group similar colors into.

    Returns:
        numpy.ndarray
            Image with indexed colors.
    """
    # Reshape image to a 2D array of pixels (rows) by color channels (columns)
    pixels = image.reshape(-1, 3)

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(pixels)

    # Get cluster centroids (colors)
    centroids = kmeans.cluster_centers_

    # Get labels for each pixel indicating the nearest centroid
    labels = kmeans.labels_

    # Replace each pixel with its nearest centroid color
    indexed_image = centroids[labels].reshape(image.shape)

    return indexed_image

# Load sample image
image = cv2.imread('logo.png')

# Apply color indexing with k-means clustering
k = 8  # Number of clusters
indexed_image = index_colors(image, k)

## Color Space Conversion

In [6]:
def convert_rgb_to_hsv(image):
    """
    Convert an image from RGB color space to HSV color space.

    Parameters:
        image: numpy.ndarray
            Input image in RGB color space.

    Returns:
        numpy.ndarray
            Image in HSV color space.
    """
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    return hsv_image

def convert_rgb_to_lab(image):
    """
    Convert an image from RGB color space to LAB color space.

    Parameters:
        image: numpy.ndarray
            Input image in RGB color space.

    Returns:
        numpy.ndarray
            Image in LAB color space.
    """
    lab_image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    return lab_image

def convert_rgb_to_ycbcr(image):
    """
    Convert an image from RGB color space to YCbCr color space.

    Parameters:
        image: numpy.ndarray
            Input image in RGB color space.

    Returns:
        numpy.ndarray
            Image in YCbCr color space.
    """
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
    return ycbcr_image

# Load sample image
image = cv2.imread('logo.png')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convert image to HSV color space
image_hsv = convert_rgb_to_hsv(image_rgb)

# Convert image to LAB color space
image_lab = convert_rgb_to_lab(image_rgb)

# Convert image to YCbCr color space
image_ycbcr = convert_rgb_to_ycbcr(image_rgb)

# Display original image and its converted versions in each color space
cv2.imshow('Original Image', image_rgb)
cv2.imshow('HSV Color Space', image_hsv)
cv2.imshow('LAB Color Space', image_lab)
cv2.imshow('YCbCr Color Space', image_ycbcr)
cv2.waitKey(0)
cv2.destroyAllWindows()

: 